In [4]:
from captum.attr import ShapleyValueSampling
from tqdm import trange

from load_data import load_data
from train_models import *
from segmentation import *
from utils import *
import torch
import os

c:\Users\Nikos\miniconda3\envs\env_segment_shap\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# device for torch
from torch.cuda import is_available as is_GPU_available
device = "cuda" if is_GPU_available() else "cpu"

# dictionary mapping predictors to torch vs other, step necessary for Captum 
predictors = {
	'torch' : ['resNet'],
	'scikit' : ['miniRocket','randomForest']
}

segmentations = {"clasp":get_claSP_segmentation, "infogain": get_InformationGain_segmentation, "greedygaussian": get_GreedyGaussian_segmentation, "equal": get_equal_segmentation, "nnsegment": get_NNSegment_segmentation}

segmentation_types = {"default", "normalized"}

In [14]:
# load data
dataset_name = 'gunpoint'
X_train, X_test, y_train, y_test, enc = load_data(subset='all', dataset_name=dataset_name)
# for debugging only
# X_test = X_test[:2]
# y_test = y_test[:2]
n_samples, n_chs, ts_length = X_test.shape

# train model
predictor_name = 'randomForest'
if predictor_name=='resNet':
	clf,preds = train_ResNet(X_train, y_train, X_test, y_test, dataset_name,device=device)
elif predictor_name=='miniRocket':
	clf,preds = train_miniRocket(X_train, y_train, X_test, y_test, dataset_name)
elif predictor_name=="randomForest":
	clf, preds = train_randomForest(X_train, y_train, X_test, y_test, dataset_name)
else:
	raise ValueError(
		"predictor not found"
	)

# segmentation
segmentation_name = "nnsegment"
segmentation_type = "normalized"
segmentation_method = segmentations[segmentation_name]

# initialize data structure meant to contain the segments
# TODO can I be cleaner here?
segments =  np.empty( (X_test.shape[0] , X_test.shape[1]), dtype=object) if X_test.shape[1] > 1  else (
	np.empty( X_test.shape[0] , dtype=object))

# create a dictionary to be dumped containing attribution and metadata
results = {
	'attributions' : {},
	'segments' : segments,
	'y_test_true' : y_test,
	'y_test_pred' : preds,
	'label_mapping' : enc,
}

training random forest
random forest accuracy is 1.0


In [15]:
# define different background to be used and number of samples as n_background
# TODO set a number of TOTAL sampling regardless of the background type?
n_background = 50
background_types = ["average", "zero","sampling"] # zero, constant, average, multisample
for bt in background_types:
	results['attributions'][bt] = np.zeros( X_test.shape ,dtype=np.float32 )

In [16]:
with torch.no_grad():
	
	SHAP = ShapleyValueSampling(clf) if predictor_name in predictors['torch'] \
		else ShapleyValueSampling(forward_classification)
	
	for i in trange ( n_samples ) : #
		
		# get current sample and label
		ts, y = X_test[i] , torch.tensor( y_test[i:i+1] )

		# get segment and its tensor representation
		current_segments = segmentation_method(ts)[:X_test.shape[1]]
		results['segments'][i] = current_segments
		mask = get_feature_mask(current_segments,ts.shape[-1])

		ts = torch.tensor(ts).repeat(1,1,1)	#TODO use something similar to np.expand_dim?

		for background_type in background_types:

			# background data
			if background_type=="zero":
				background_dataset = torch.zeros((1,) + X_train.shape[1:])
			elif background_type=="sampling":
				background_dataset = sample_background(X_train, n_background)
			elif background_type=="average":
				background_dataset = sample_background(X_train, n_background).mean(axis=0, keepdim=True)

			# for sampling strategy repeat the ts many times as the background dataset size
			ts = ts.repeat(background_dataset.shape[0],1,1) if background_type=="sampling" else ts

			# different call depending on predictor type
			if predictor_name in predictors['scikit']:
				# if using random forest flat everything
				if predictor_name=="randomForest":
					ts = ts.reshape( -1, n_chs*ts_length); mask = mask.reshape( -1, n_chs*ts_length);
					background_dataset = background_dataset.reshape( -1, n_chs*ts_length)
				
				tmp = SHAP.attribute( ts, target=y , feature_mask=mask, baselines=background_dataset, additional_forward_args=clf)
			
			elif predictor_name in predictors['torch']:
				# if use torch make sure everything is on selected device
				ts = ts.to(device); y = y.to(device)
				mask = mask.to(device) ; background_dataset =  background_dataset.to(device)
				tmp = SHAP.attribute( ts, target=y , feature_mask=mask, baselines=background_dataset)
			
			# 'un-flatten' for randomForest
			if predictor_name=="randomForest":
				tmp = tmp.reshape(-1,X_test.shape[1],X_test.shape[2])

			# store current explanation in the data structure; if sampling store the mean
			results['attributions'][background_type][i] = torch.mean(tmp, dim=0).cpu().numpy() if \
				background_type=="sampling" else tmp[0].cpu().numpy()

	if segmentation_type=="normalized":
		weights = np.array(list(map(lambda x: list(map(lambda y: lengths_to_weights(change_points_to_lengths(y, X_train.shape[-1])), x)), results["segments"])))
		for background_type in background_types:
			results["attributions"][background_type] *= weights

		


  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\Nikos\miniconda3\envs\env_segment_shap\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Nikos\miniconda3\envs\env_segment_shap\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\Nikos\miniconda3\envs\env_segment_shap\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\Nikos\miniconda3\envs\env_segment_shap\Lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
c:\Users\Nikos\miniconda3\envs\env_segment_shap\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / r

[0 15 31 47 101 116] [15 31 47 101 116 150]
[0 43 63 84 99 117] [43 63 84 99 117 150]


In [ ]:
# dump result to disk
file_name = "_".join ( ( predictor_name, dataset_name ) )+".npy"
file_path = os.path.join("attributions", file_name)
np.save( file_path, results )